In [1]:
import numpy as np
import pandas as pd
import os

from analysis import plot
from bmtool.util import util

network_name = 'v1'

### File paths

In [2]:
model_dir = os.path.join('..', 'V1_bio')
data_dir = os.path.join('.', 'data')

nodes_file = os.path.join(model_dir, 'network', 'v1_nodes.h5')
node_types_file = os.path.join(model_dir, 'network', 'v1_node_types.csv')

### Load nodes

In [3]:
v1_nodes = util.load_nodes_from_paths([{"nodes_file": nodes_file, "node_types_file": node_types_file}])[network_name]
# population properties
v1_pop_df = v1_nodes[['pop_name', 'model_type', 'ei', 'location']].set_index('pop_name')
v1_pop_df = v1_pop_df[~v1_pop_df.index.duplicated(keep='first')]
# update pop_name
v1_pop_df['pop_name'] = v1_pop_df.index
for p, row in v1_pop_df.iterrows():
    row['location'] = row['location'][4:]
    if row['ei'] == 'e':
        name = 'e' + row['location']
        row['pop_name'] = p[:p.find(name) + len(name)]
v1_nodes['pop_name'] = v1_pop_df.loc[v1_nodes['pop_name'], 'pop_name'].values
v1_pop_df.set_index('pop_name', inplace=True)
v1_pop_df = v1_pop_df[~v1_pop_df.index.duplicated(keep='first')]
v1_pop_df.rename(columns={'location': 'layer'}, inplace=True)
# add population properties
v1_pop_df[['cell_type', 'color']] = ''
for p, row in v1_pop_df.iterrows():
    row['cell_type'] = p[len(row.layer) + (1 if row.model_type == 'biophysical' else 4):] if row.ei == 'i' else 'e'
    row['color'] = plot.pop_color[row.cell_type].replace(' ' if row.model_type == 'biophysical' else 'tab:', '')

#### Save nodes information to data folder

In [4]:
v1_pop_df.to_csv(os.path.join(data_dir, 'v1_population_properties.csv'))
bio_idx = v1_pop_df['model_type'] == 'biophysical'
bio_pop_names = v1_pop_df.index[bio_idx].tolist()
lif_pop_names = v1_pop_df.index[~bio_idx].tolist()
display(v1_pop_df.loc[bio_pop_names])
display(v1_pop_df.loc[lif_pop_names])

,model_type,ei,layer,cell_type,color
pop_name,,,,,
i1Htr3a,biophysical,i,1,Htr3a,tab:purple
e23,biophysical,e,23,e,tab:red
i23Htr3a,biophysical,i,23,Htr3a,tab:purple
i23Pvalb,biophysical,i,23,Pvalb,tab:blue
i23Sst,biophysical,i,23,Sst,tab:green
e4,biophysical,e,4,e,tab:red
i4Htr3a,biophysical,i,4,Htr3a,tab:purple
i4Pvalb,biophysical,i,4,Pvalb,tab:blue
i4Sst,biophysical,i,4,Sst,tab:green


,model_type,ei,layer,cell_type,color
pop_name,,,,,
LIFi1Htr3a,point_process,i,1,Htr3a,purple
LIFe23,point_process,e,23,e,red
LIFi23Htr3a,point_process,i,23,Htr3a,purple
LIFi23Pvalb,point_process,i,23,Pvalb,blue
LIFi23Sst,point_process,i,23,Sst,green
LIFe4,point_process,e,4,e,red
LIFi4Htr3a,point_process,i,4,Htr3a,purple
LIFi4Pvalb,point_process,i,4,Pvalb,blue
LIFi4Sst,point_process,i,4,Sst,green


In [5]:
bio_idx = v1_nodes['model_type'] == 'biophysical'
v1_bio_nodes = v1_nodes[bio_idx].copy() # choose biophysical neurons
v1_lif_nodes = v1_nodes[~bio_idx].copy() # choose point neurons

# IDs for raster plot in a reverse order of layers
v1_bio_nodes['raster_id'] = v1_bio_nodes.index
n = len(v1_bio_nodes)
for p in bio_pop_names:
    idx = v1_bio_nodes['pop_name'] == p
    n_pop = idx.sum()
    v1_bio_nodes.loc[idx, 'raster_id'] = range(n - n_pop, n)
    n -= n_pop

v1_lif_nodes['raster_id'] = v1_lif_nodes.index
n = len(v1_lif_nodes)
for p in lif_pop_names:
    idx = v1_lif_nodes['pop_name'] == p
    n_pop = idx.sum()
    v1_lif_nodes.loc[idx, 'raster_id'] = range(n - n_pop, n)
    n -= n_pop

v1_bio_nodes.to_csv(os.path.join(data_dir, 'v1_biophysical_nodes.csv'))
v1_lif_nodes.to_csv(os.path.join(data_dir, 'v1_point_process_nodes.csv'))